#Exoplanets Hunting with Machine Learning

 An exoplanet is any planet beyond our solar system. They are detected by astronomers by observing the intensity of their parent starts.

 Exoplanetsare made up of elements similar to those of the planets in our solar system, but their mixes of those elements may differ.

 Exoplanets come in a wide variety pf sizes, from gas giants larger than jupiter to small, rocky planets about as big around as Earth ot Mars.



**History**

` 1. Early claims to have detected exoplanets were made in the 19th century, notably by William Jacob.`

`2. In 1855, William saw deviations in the orrbits within the nearby 70 ophiuchi binary start system.`

`3. The first time we actually detected an exoplanet was in 1917 NASA's JPL but wasn't confirmed.`